In [ ]:
import numpy as np
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split


from mto import MTO



# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Preprocess the data
x_train = x_train.reshape(-1, 784).astype('float32') / 255.0
x_test = x_test.reshape(-1, 784).astype('float32') / 255.0
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

# Define the model architecture
def create_model(n_layers, n_units, dropout_rate):
    model = Sequential()
    for i in range(n_layers):
        if i == 0:
            model.add(Dense(n_units, activation='relu', input_shape=(784,)))
        else:
            model.add(Dense(n_units, activation='relu'))
        model.add(Dropout(dropout_rate))
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Define the objective function to be optimized
def objective_function(x):
    n_layers = int(np.round(x[0]))
    n_units = int(np.round(x[1]))
    dropout_rate = x[2]
    model = create_model(n_layers, n_units, dropout_rate)
    model.fit(x_train, y_train, batch_size=128, epochs=10, verbose=0)
    loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
    return -accuracy  # We maximize accuracy, so we need to minimize the negative of accuracy

# Set the bounds for the variables to be optimized
bounds = [(1, 5), (32, 256), (0, 0.5)]

# Create an instance of the MTO algorithm
mto = MTO(objective_function, bounds, population_size=50, max_iter=70)

print("calling mto")
# Run the MTO algorithm
best_solution = mto.run()


# Print the best solution found
print('Best solution found: {}'.format(best_solution))


calling mto
